In [2]:
Work.objects.count()

15177

In [7]:
NB_WORKS_LIMIT = 20
NB_USERS_LIMIT = 200

SearchIssue.objects.all().delete()
# Profile.objects.all().delete()  # Keeping some profiles would be better
Suggestion.objects.all().delete()
Recommendation.objects.all().delete()
Pairing.objects.all().delete()
SocialApp.objects.all().delete()
Session.objects.all().delete()
ContentType.objects.all().delete()
# Reference.objects.all().delete()

(0, {})

In [8]:
from collections import Counter
nb = Counter(Rating.objects.values_list('work_id', flat=True))
if NB_WORKS_LIMIT:
    work_ids = list(Work.objects.values_list('id', flat=True))
    work_ids.sort(key=lambda work_id: -nb[work_id])
    Work.objects.exclude(id__in=work_ids[:NB_WORKS_LIMIT]).delete()

In [9]:
if NB_WORKS_LIMIT:
    bundle = []
    for artist_ids in Work.objects.values_list('artists'):
        bundle.extend(artist_ids)
    kept_artist_ids = list(set([x for x in bundle if x is not None]))
    Artist.objects.exclude(id__in=kept_artist_ids).delete()

In [10]:
Artist.objects.count()

42

In [11]:
Work.objects.count()

20

In [12]:
Rating.objects.count()

23236

In [13]:
max_user_id = max(User.objects.values_list('id', flat=True))

In [14]:
import random
chosen = User.objects.order_by('?')[:NB_USERS_LIMIT]
kept_ids = chosen.values_list('id', flat=True)
User.objects.exclude(id__in=kept_ids).delete()

(47007,
 {'account.EmailAddress': 1925,
  'account.EmailConfirmation': 1867,
  'admin.LogEntry': 581,
  'auth.User': 1942,
  'auth.User_groups': 2,
  'auth.User_user_permissions': 0,
  'irl.Attendee': 10,
  'mangaki.Neighborship': 17460,
  'mangaki.Pairing': 0,
  'mangaki.Profile': 1940,
  'mangaki.Rating': 21179,
  'mangaki.Recommendation': 0,
  'mangaki.SearchIssue': 0,
  'socialaccount.SocialAccount': 101,
  'socialaccount.SocialToken': 0})

In [15]:
User.objects.count()

200

In [16]:
from django.utils import timezone
new_ids = random.sample(range(max_user_id + 1, max_user_id + 201), 200)
for user, new_id in zip(User.objects.all(), new_ids):
    old_id = user.id
    user.pk = new_id
    user.username = str(new_id)
    user.is_superuser = False
    user.set_password('mangaki')
    user.email = '%d@mangaki.fr' % new_id
    user.date_joined = timezone.now()
    user.last_login = timezone.now()
    user.save()
    old_user = User.objects.get(id=old_id)
    old_user.rating_set.update(user=user)
    # old_user.profile.user = user
    old_user.delete()

In [17]:
User.objects.all()

<QuerySet [<User: 2195>, <User: 2212>, <User: 2288>, <User: 2353>, <User: 2237>, <User: 2318>, <User: 2259>, <User: 2316>, <User: 2293>, <User: 2266>, <User: 2307>, <User: 2311>, <User: 2159>, <User: 2292>, <User: 2221>, <User: 2277>, <User: 2169>, <User: 2285>, <User: 2333>, <User: 2240>, '...(remaining elements truncated)...']>

Now, please make ``./mangaki/manage.py dumpdata > fixtures/seed_data.json``

In [21]:
from collections import Counter
import json
j = json.load(open('../fixtures/seed_data.json'))
# Currently
"""Counter({'auth.group': 1,  # May be removed
         'auth.user': 200,
         'irl.event': 38,     # May be removed
         'irl.location': 2,   # May be removed
         'irl.partner': 12,   # May be removed
         'mangaki.artist': 42,
         'mangaki.artistspelling': 42,
         'mangaki.category': 3,
         'mangaki.editor': 167,
         'mangaki.genre': 34,
         'mangaki.page': 4,
         'mangaki.profile': 200,
         'mangaki.rating': 2057,
         'mangaki.reference': 29,
         'mangaki.role': 5,
         'mangaki.staff': 60,
         'mangaki.studio': 50,
         'mangaki.top': 4,
         'mangaki.work': 20,
         'sites.site': 1})    # May be removed"""
Counter(entry['model'] for entry in j)

Counter({'auth.user': 200,
         'irl.event': 38,
         'irl.location': 2,
         'irl.partner': 12,
         'mangaki.artist': 48,
         'mangaki.artistspelling': 48,
         'mangaki.category': 3,
         'mangaki.editor': 24,
         'mangaki.genre': 34,
         'mangaki.page': 4,
         'mangaki.rating': 3161,
         'mangaki.reference': 30,
         'mangaki.role': 5,
         'mangaki.staff': 100,
         'mangaki.studio': 50,
         'mangaki.top': 3,
         'mangaki.work': 40})